# Reproduce Guinness example With OSRM docker image

To utilize the OSRM routing engine, you should take a moment to set up the OSRM docker image on your machine. While this can be a little tedious to set up, this allows the `route` module to quickly identify shortest routes between points of interest and solve VRP problems.

The first step to do this is to obtain the 'raw' data for the area in which you are operating. This takes the form of `.pbf` files, which can be obtained from the [geofabrik](https://download.geofabrik.de/_) portal. In our case, we obtain the [Ireland and Northern Ireland](https://download.geofabrik.de/europe/ireland-and-northern-ireland.html) `.pbf` file.

Once downloaded, the `.pbf` needs to be processed using a series of extraction, partitioning, and customization commands. This is easiest to do as a shell script. The version of this script for this example can be found in `PATH/osrm.sh`, it looks like this:

This script will create several additional files from the `.pbf` that you downloaded, which are required for identifying shortest routes and solving the VRP using real street network data.

Finally, the OSRM backend docker container should obtained and activated. While you do have the option to compile OSRM locally in C++, the docker container is a much simpler way to access the service. Find the latest release of the OSRM docker container at the [osrm-backend github repository](https://github.com/Project-OSRM/osrm-backend).

This container should be 'spun up' as a service, meaning it holds a port on your machine available to hear requests and send those requests to the OSRM servers to obtain routes, distances, and durations. The `spopt.route` module will then take those data and solve the defined VRP. Once you have the docker image in your directory, start the service by running the following command in your terminal: 

`docker run -t -i -p 5000:5000 -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-routed --algorithm mld --max-table-size 9999999 /data/ireland-and-northern-ireland-latest.osrm`

Let's breakdown this command. This part:

`docker run -t -i -p 5000:5000 -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-routed`

says we're connecting to the docker service on the 5000 port. We're porting into the data directory of the backend and starting the osrm-routed service. 

This part:
 
`--algorithm mld --max-table-size 9999999 /data/ireland-and-northern-ireland-latest.osrm`

says that we're going to use the MLD (Multi-Level Dijkstra) algorithm to identify the most optimal routes in our problem. the `--max-table-size 99999999` argument indicates we are increasing the rate limitations for problem size, which is quite low by default. Finally, `data/ireland-and-northern-ireland-latest.osrm` is where our processed data file is, which tells OSRM where we're trying to operate. 

After running this final command, the service is activated and listening for requests in your terminal. 

In [1]:
import geopandas as gpd
import pandas, numpy, pyvrp, sys

In [2]:
sys.path.insert(0, '/home/dylan/projects/gsoc2025/spopt/') # not the published spopt

In [3]:
import spopt
print(spopt.__file__)

/home/dylan/projects/gsoc2025/spopt/spopt/__init__.py


In [4]:
from spopt.route import engine, heuristic, utils

In [5]:
from spopt.route.heuristic import LastMile
from pyvrp import stop

## Reproduce the guinness example

In [6]:
trucks = pandas.DataFrame(
    [['big', 'lng',      2000,    280, .004,  .50, 5],
     ['big', 'electric', 2000,    480, .002,  .50, 5],
     ['med', 'lng',      800, 280*.66, .0001, .63, 10],
     ['med', 'electric', 800, 480*.66, .004,  .50, 10],
     ['smo', 'lng',      400, 280*0.4, .002,  .50, 20],
     ['smo', 'electric', 400, 480*0.4, .0001, .63, 20],
     ],
     columns = [
         'namesize', 'namefuel', 'capacity', 
         'fixed_cost', 'cost_per_meter', 'cost_per_minute', 'n_truck'
         ]
)

In the code snippet above, we define a DataFrame quantifying the available fleet of trucks. Each row represents a different truck type, identified by size (`namesize`) and fuel type (`namefuel`). The `capacity` column indicates how much the truck can carry. `fixed_cost` is the base cost of using the truck, regardless of how far it travels. `cost_per_meter` and `cost_per_minute` represent variable costs that depend on distance and travel time. The `n_truck` column tells us how many of each truck type are available. 

Now we need to obtain the file for the depot and clients for this problem.

(make available as a function? e.g. route.example()?)

In [7]:
dublin_pubs = gpd.read_file('/home/dylan/projects/gsoc2025/spopt/notebooks/gsoc2025/data/dublinpubs.geojson')

In [8]:
dublin_pubs.shape

(551, 8)

These are the clients and vehicle depot for our Vehicle Routing Problem. The file contains pubs all across Dublin, stored in a GeoJSON file and read into a GeoDataFrame using GeoPandas. Each row in this table represents a location, either a pub (client) or the Guinness Storehouse (the depot), with associated geographic coordinates and attributes relevant to the routing problem.

In [9]:
gdf = dublin_pubs

In [10]:
clients = gdf.iloc[1:,:].reset_index(drop=True)
clients = clients.set_index(clients.osmid.astype(str))

In [11]:
clients.head()

,osmid,name,longitude,latitude,factype,demand,supply,geometry
osmid,,,,,,,,
133193958,133193958,The Barge Bar,-6.260588,53.330551,client,23.0,21.0,POINT (-6.26059 53.33055)
249426205,249426205,Marino House (Kavanagh's),-6.228816,53.365756,client,16.0,12.0,POINT (-6.22882 53.36576)
279464425,279464425,The Lower Deck,-6.265238,53.330686,client,28.0,5.0,POINT (-6.26524 53.33069)
279915988,279915988,Arlington Bar & Restaurant,-6.260933,53.347252,client,16.0,11.0,POINT (-6.26093 53.34725)
280123269,280123269,Bar@Tolka,-6.251268,53.367676,client,26.0,16.0,POINT (-6.25127 53.36768)


Clients have associated `demand` and `supply` values, which represent how many kegs needs to be delivered to or picked up from that site. The `geometry` column stores the location as a geographic point. 

In [12]:
depot = gdf.iloc[0,:]

We extract the first row as the depot, which serves as the start and end point for all vehicle routes. The remaining rows are identified as clients and indexed by their unique IDs from Openstreetmap.

Now we have all the tools to set up and solve a Vehicle Routing Problem. First, we initalize the problem, setting the depot location and optionally setting the operating hours.

In [13]:
print('initializing model')
m = LastMile(
    depot_location=(depot.longitude.item(), depot.latitude.item()),
    depot_open=pandas.Timestamp("2030-01-02 07:00:00"),
    depot_close=pandas.Timestamp("2030-01-02 20:00:00"),
    depot_name=depot['name'],
)

initializing model


Then, we add the clients to be serviced in the routing problem.

In [14]:
print("adding clients")
m.add_clients(
    locations = clients.geometry, 
    delivery = clients.demand,
    pickup = clients.supply,
    time_windows=None,
    service_times=(numpy.log(clients.demand)**2).astype(int)
)

adding clients


Lastly, we add the available delivery vehicles to the model object.

In [15]:
print("adding trucks")
m.add_trucks_from_frame(
    trucks, 
)

adding trucks


| Router       | Required Keywords | API key? | Requires Backend? | confirmed functional | 
|--------------|----------|----------| ----------------  | ------------------- |
| OSRM         | base-url | no | yes     | yes | 
| Valhalla | base-url, profile | no | yes | no | 
| HereMaps     |          | Yes      | | no |
| Google       |          | Yes      | | no |
| Graphhopper  |          | Yes      | | no |
| Mapbox OSRM  |          | Yes      | | no |
| OpenRouteService |      | Yes      | | no |
| OpenTripPlanner | | | | N/A (`matrix` not implemented) | 

In [16]:
from routingpy import OSRM

In [17]:
m.solve(stop=pyvrp.stop.MaxRuntime(10), routing=OSRM, routing_kws={"base_url": "http://localhost:5000"})

routing engine is defined as: <routingpy.routers.osrm.OSRM object at 0x7f118c62bbf0>
PyVRP v0.11.3

Solving an instance with:
    1 depot
    550 clients
    70 vehicles (6 vehicle types)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best

Search terminated in 10.12s after 19 iterations.
Best-found solution has cost 44679994.

Solution results
    # routes: 15
     # trips: 15
   # clients: 550
   objective: 44679994
    distance: 336715
    duration: 1584
# iterations: 19
    run-time: 10.12 seconds

[routes_and_stops] routing passed type: <class 'routingpy.routers.osrm.OSRM'>
[routes_and_stops] routing passed type: <class 'routingpy.routers.osrm.OSRM'>
[routes_and_stops] routing passed type: <class 'routingpy.routers.osrm.OSRM'>
[routes_and_stops] routing passed type: <class 'routingpy.routers.osrm.OSRM'>
[routes_and_stops] routing passed type: <class 'routingpy.routers.osrm.OSRM'>
[routes_and_stops] routing 

In [18]:
m.write_result("osrm")

In [19]:
m.solve(stop=pyvrp.stop.MaxRuntime(10))

/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:118: UserWarning: Failed to connect to routing engine... using haversine distancedurations = 0
  warnings.warn(


PyVRP v0.11.3

Solving an instance with:
    1 depot
    550 clients
    70 vehicles (6 vehicle types)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best

Search terminated in 10.39s after 19 iterations.
Best-found solution has cost 174870190.

Solution results
    # routes: 69
     # trips: 69
   # clients: 550
   objective: 174870190
    distance: 1772205
    duration: 550
# iterations: 19
    run-time: 10.39 seconds

[routes_and_stops] routing passed type: <class 'NoneType'>
[routes_and_stops] routing passed type: <class 'NoneType'>
[routes_and_stops] routing passed type: <class 'NoneType'>
[routes_and_stops] routing passed type: <class 'NoneType'>
[routes_and_stops] routing passed type: <class 'NoneType'>
[routes_and_stops] routing passed type: <class 'NoneType'>
[routes_and_stops] routing passed type: <class 'NoneType'>
[routes_and_stops] routing passed type: <class 'NoneType'>
[routes_and_stops] routing p

In [20]:
m.write_result("no-routing-engine")

In [ ]:
from routingpy import Valhalla

In [ ]:
m.solve(stop=pyvrp.stop.MaxRuntime(10), routing=Valhalla, routing_kws={"base_url": "http://localhost:8002", "profile": "auto"})

In [ ]:
m.write_results("valhalla")

In [ ]:
import shapely

In [ ]:
all_lonlats = numpy.vstack(
    [m.depot_location] + list(shapely.get_coordinates(m.clients_.geometry))
)

In [ ]:
from routingpy import HereMaps

In [ ]:
m.solve(stop=pyvrp.stop.MaxRuntime(60), routing=HereMaps, routing_kws={})

In [ ]:
from routingpy import Graphhopper

In [ ]:
m.solve(stop=pyvrp.stop.MaxRuntime(60), routing=Graphhopper, routing_kws={})

In [ ]:
from routingpy import MapboxOSRM

In [ ]:
m.solve(stop=pyvrp.stop.MaxRuntime(60), routing=MapboxOSRM, routing_kws={})

In [ ]:
from routingpy import ORS

In [ ]:
m.solve(stop=pyvrp.stop.MaxRuntime(60), routing=ORS, routing_kws={})

In [ ]:
from routingpy import OpenTripPlannerV2

In [ ]:
OpenTripPlannerV2.matrix?
# not implemented (https://github.com/mthh/routingpy/blob/master/routingpy/routers/opentripplanner_v2.py#L323)